In [1]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np
import pandas as pd
import cv2
import os
import h5py
import pickle

/home/ali/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from  sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from catboost import CatBoostClassifier
from  xgboost import XGBClassifier

from sklearn.metrics import (classification_report,accuracy_score,confusion_matrix,
                             precision_score, recall_score, f1_score)

In [3]:
X = np.array(h5py.File(os.path.join('data','X.h5'),'r')['features'])
y = np.array(h5py.File(os.path.join('data','y.h5'),'r')['targets'])

In [4]:
np.shape(X)

(1360, 1019)

In [5]:
X

array([[3.13705759e-01, 5.55847071e-02, 0.00000000e+00, ...,
        5.79671025e-01, 5.81509021e-01, 5.83351017e-01],
       [4.13660545e-01, 2.76928505e-02, 4.24824605e-05, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [8.07397387e-01, 1.49239223e-01, 0.00000000e+00, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       ...,
       [2.63134608e-01, 1.06193822e-01, 1.27563254e-01, ...,
        9.56468847e-02, 9.57946124e-02, 9.59451801e-02],
       [5.84949132e-04, 1.38654032e-03, 4.32320717e-02, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [9.12752606e-05, 7.32482080e-03, 2.96973376e-03, ...,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00]])

In [34]:
X_train, X_test, y_train ,y_test = train_test_split(X,y,test_size = 0.2,random_state=100)

In [35]:
# dtc = DecisionTreeClassifier()
knnc = KNeighborsClassifier(n_neighbors=15,n_jobs=-1)
rfc = RandomForestClassifier(n_jobs=-1,n_estimators=60000,
                             max_depth=16,min_samples_split= 2)
# abc = AdaBoostClassifier(learning_rate=.85,n_estimators=3000,)
cbc = CatBoostClassifier(iterations=2000, learning_rate=.1,depth=16,verbose=100)
gbc = GradientBoostingClassifier(learning_rate= 0.8, max_depth= 15, max_features= 8, min_samples_split= 8)
# xg_reg  =XGBClassifier(max_depth=15,min_child_weight= 1,gamma= 0.5, learning_rate= 0.2,n_jobs=-1)
# class_list=[svc,dtc,knnc,rfc,abc,gbc,xg_reg]
class_list=[rfc]

In [36]:
%%time

counter=1
for reg in class_list:
    reg.fit(X_train,y_train)
    print(f"Fitting {counter} done")
    counter+=1

Fitting 1 done
CPU times: user 25min 43s, sys: 47.7 s, total: 26min 31s
Wall time: 5min 20s


In [37]:
%%time
score_dict={}

for reg in class_list:
    
    reg_name = reg.__class__.__name__
    
    pred=reg.predict(X_test)
    
    print("-"*50)
    
    print(reg_name)
  
    print(f"accuracy score : {accuracy_score(y_test,pred)}")
    print(f"classification report : {classification_report(y_test,pred)}")


    score_dict[reg_name]={
        "accuracy score":accuracy_score(y_test,pred),
        "classification report" :classification_report(y_test,pred),

    }

--------------------------------------------------
RandomForestClassifier
accuracy score : 0.6433823529411765
classification report :               precision    recall  f1-score   support

           0       0.56      0.75      0.64        12
           1       0.58      0.41      0.48        17
           2       0.56      0.74      0.64        19
           3       0.78      0.74      0.76        19
           4       0.56      0.74      0.64        19
           5       0.50      0.38      0.43        13
           6       0.83      0.91      0.87        22
           7       0.67      0.56      0.61        18
           8       1.00      0.33      0.50        21
           9       0.70      1.00      0.82         7
          10       0.62      0.53      0.57        19
          11       0.62      0.42      0.50        19
          12       0.61      0.85      0.71        13
          13       0.81      0.93      0.87        14
          14       0.42      0.56      0.48         9
 

In [24]:
score_pd=pd.DataFrame(score_dict).transpose().sort_values("accuracy score" ,ascending=False)
score_pd

,accuracy score,classification report
RandomForestClassifier,0.639706,precision recall f1-score ...
KNeighborsClassifier,0.194853,precision recall f1-score ...
GradientBoostingClassifier,0.088235,precision recall f1-score ...


In [10]:
score_pd=pd.DataFrame(score_dict).transpose().sort_values("accuracy score" ,ascending=False)
score_pd

,accuracy score,classification report
RandomForestClassifier,0.628676,precision recall f1-score ...
KNeighborsClassifier,0.283088,precision recall f1-score ...
GradientBoostingClassifier,0.121324,precision recall f1-score ...


In [12]:
# rf = RandomForestClassifier(n_estimators=400)
# svc = SVC()

In [13]:
# rf.fit(X_train,y_train)
# svc.fit(X_train,y_train)

In [14]:
# y_pred_rf = rf.predict(X_test)
# # y_pred_svc = svc.predict(X_test)

In [15]:
# print(classification_report(y_pred=y_pred_rf, y_true= y_test))

In [16]:
# print(classification_report(y_pred=y_pred_svc, y_true= y_test))

In [17]:
# model = CatBoostClassifier(iterations=200, learning_rate=0.05, depth=10,verbose=20)


In [18]:
# model.fit(X_train,y_train)

In [19]:
# y_pred = model.predict(X_test)

In [20]:
# print(classification_report(y_pred=y_pred, y_true= y_test))